# EX01-EXTRA — IBM HR Data Analysis (Pandas)

**Objective:** practice Pandas on a real dataset and produce meaningful insights.
**Dataset:** IBM HR Analytics Employee Attrition & Performance.

**Note:** if the file is not found, download it from Kaggle and place it next to the notebook.


# Import


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path


## 1. Load data
Place the CSV next to the notebook or set the path manually.


In [ ]:
# Optional override (set a path string if needed)
path_override = None  # e.g. '/content/drive/MyDrive/ibm_hr.csv'

candidate_paths = [
    'WA_Fn-UseC_-HR-Employee-Attrition.csv',
    'data/WA_Fn-UseC_-HR-Employee-Attrition.csv',
    '/content/WA_Fn-UseC_-HR-Employee-Attrition.csv',
]

path = path_override or next((p for p in candidate_paths if Path(p).exists()), None)
if path is None:
    raise FileNotFoundError('Dataset not found. Download the CSV from Kaggle and place it next to the notebook or in data/, or set path_override.')
df = pd.read_csv(path)


## 2. Initial inspection
Check shape, dtypes, and missing values.


In [ ]:
df.shape


In [ ]:
df.dtypes


In [ ]:
df.isna().sum().sort_values(ascending=False).head(20)


Problem type: **binary classification**.\nSuggested target: **Attrition** (left vs stayed).\n

## 3. Data preparation
Split features, drop constants, create a derived feature.


In [ ]:
num_cols = df.select_dtypes(include='number').columns.tolist()
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
num_cols, cat_cols


In [ ]:
# Drop constant columns
constant_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
df_prep = df.drop(columns=constant_cols)
constant_cols


In [ ]:
# Derived feature: tenure group (fallback to age if YearsAtCompany is missing)
if 'YearsAtCompany' in df_prep.columns:
    df_prep['TenureGroup'] = pd.cut(
        df_prep['YearsAtCompany'],
        bins=[-1, 2, 5, 10, 20, np.inf],
        labels=['0-2', '3-5', '6-10', '11-20', '20+']
    )
elif 'Age' in df_prep.columns:
    df_prep['AgeGroup'] = pd.cut(
        df_prep['Age'],
        bins=[0, 24, 34, 44, 54, np.inf],
        labels=['<25', '25-34', '35-44', '45-54', '55+']
    )


## 4. Data analysis
Answer 3+ questions from the task (groupby, aggregates, filtering).


In [ ]:
# Attrition vs income and satisfaction (if columns exist)
if 'Attrition' in df_prep.columns:
    cols = [c for c in ['MonthlyIncome', 'JobSatisfaction'] if c in df_prep.columns]
    if cols:
        display(df_prep.groupby('Attrition')[cols].agg(['mean', 'median', 'count']))


In [ ]:
# Department/Role attrition rate
if 'Attrition' in df_prep.columns:
    if 'Department' in df_prep.columns:
        dept_rate = (
            df_prep.groupby('Department')['Attrition']
            .apply(lambda s: (s == 'Yes').mean())
            .sort_values(ascending=False)
        )
        display(dept_rate)
    if 'JobRole' in df_prep.columns:
        role_rate = (
            df_prep.groupby('JobRole')['Attrition']
            .apply(lambda s: (s == 'Yes').mean())
            .sort_values(ascending=False)
        )
        display(role_rate.head(10))


In [ ]:
# OverTime vs Attrition
if {'OverTime', 'Attrition'}.issubset(df_prep.columns):
    overtime_rate = (
        df_prep.groupby('OverTime')['Attrition']
        .apply(lambda s: (s == 'Yes').mean())
        .sort_values(ascending=False)
    )
    display(overtime_rate)


In [ ]:
# Tenure group vs income
if 'TenureGroup' in df_prep.columns and 'MonthlyIncome' in df_prep.columns:
    display(df_prep.groupby('TenureGroup')['MonthlyIncome'].agg(['mean', 'median', 'count']))


## 5. Summary
Key observations and ML problem framing.
**Write 2–3 observations**, highlight informative features, and state the ML task type.
